In [22]:
import os
import pandas as pd
import nltk
nltk.download('wordnet')
wn = nltk.corpus.wordnet
from digital_manuscript import BnF

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\danac\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [23]:
path = os.getcwd() + '/../manuscript-object/thesaurus/animal.csv'

df = pd.read_csv(path)
terms = list(set(df['prefLabel_en']))
simple_terms = [t for t in terms if ' ' not in t]
simple_terms

['passerine',
 'human',
 'tellin',
 'bird',
 'dog',
 'horse',
 'cicada',
 'pig',
 'lamb',
 'turtledofe',
 'cuttlefish',
 'crayfish',
 'ortolan',
 'water-dog',
 'calendra',
 'hart',
 'codfish',
 'mule',
 'hen',
 'bittern',
 'eagle',
 'grasshopper',
 'dragon',
 'fish',
 'lizard',
 'verdaule',
 'cuckoo',
 'turtle',
 'nightingale',
 'barbel',
 'bat',
 'rabbit',
 'petit-gri',
 'worm',
 'ox',
 'siskin',
 'lark',
 'wolf',
 'mutton',
 'swine',
 'hog',
 'earthworm',
 'frog',
 'chicken',
 'beef',
 'mouse',
 'daot',
 'linnet',
 'ewe',
 'cat',
 'duck',
 'pork',
 'bombicum',
 'silkworm',
 'animal',
 'ant',
 'fly',
 'mealworm',
 'snake',
 'toad',
 'weasel',
 'butterfly',
 'mussel',
 'chaffinch',
 'swan',
 'chick',
 'snail',
 'shell',
 'calandra',
 'flea',
 'crab',
 'rat',
 'calf',
 'partridge',
 'cow',
 'dormouse',
 'sheep',
 'goat',
 'crow',
 'swallow',
 'squirrel',
 'dogfish',
 'porcupine',
 'louse',
 'goldfinch',
 'og',
 'oyster',
 'viper',
 'aucupio',
 'pigeon',
 'colt',
 'spider']

In [24]:
term = 'dog'
dog = wn.synsets(term, pos=wn.NOUN)
dog
# hyper = dog[0].hypernyms()
# for x in hyper:
#     print(x, x.hypernyms())

# from nltk.corpus import wordnet as wn
# for ss in wn.synsets(term):
#     print (ss, ss.hypernyms())

[Synset('dog.n.01'),
 Synset('frump.n.01'),
 Synset('dog.n.03'),
 Synset('cad.n.01'),
 Synset('frank.n.02'),
 Synset('pawl.n.01'),
 Synset('andiron.n.01')]

In [25]:
animal = wn.synset('animal.n.01')

class r_node:
    def __init__(self, lemma, children=[]):
        self.lemma = lemma
        
#         if lemma == animal:
#             print(children + [lemma,])
        
        self.children = children
        self.hypernyms = lemma.hypernyms()
        self.parents = [r_node(h, children + [lemma,]) for h in self.hypernyms]
        
    def return_animal_path(self):
        if self.lemma == animal:
            return [[self.lemma,],]
        else:
            parent_paths = [n.return_animal_path() for n in self.parents]
            parent_paths = [p for p in parent_paths if len(p) > 0]
            paths = []
            for p in parent_paths:
                paths += p
            for p in paths:
                p.append(self.lemma)
            return paths

In [26]:
dog = wn.synsets('dog', pos=wn.NOUN)[0]
dn = r_node(dog)
dn.return_animal_path()
# print([p.parents[0].lemma for p in dn.parents])

# animal


[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('carnivore.n.01'),
  Synset('canine.n.02'),
  Synset('dog.n.01')],
 [Synset('animal.n.01'), Synset('domestic_animal.n.01'), Synset('dog.n.01')]]

In [27]:
squirrel = wn.synsets('squirrel')[0]
sn = r_node(squirrel)
sn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('mammal.n.01'),
  Synset('placental.n.01'),
  Synset('rodent.n.01'),
  Synset('squirrel.n.01')]]

In [28]:
fish = wn.synsets('fish', pos=wn.NOUN)[0]
fn = r_node(fish)
fn.return_animal_path()

[[Synset('animal.n.01'),
  Synset('chordate.n.01'),
  Synset('vertebrate.n.01'),
  Synset('aquatic_vertebrate.n.01'),
  Synset('fish.n.01')]]

In [29]:
louse = wn.synsets('louse')[0]
ln = r_node(louse)
ln.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('arthropod.n.01'),
  Synset('insect.n.01'),
  Synset('louse.n.01')]]

In [30]:
oyster = wn.synsets('oyster')[0]
on = r_node(oyster)
on.return_animal_path()

[[Synset('animal.n.01'),
  Synset('invertebrate.n.01'),
  Synset('mollusk.n.01'),
  Synset('bivalve.n.01'),
  Synset('oyster.n.01')]]

In [31]:
def get_animal_paths(lemma):
    paths = []
    node = r_node(lemma)
    
    
    
    

In [32]:
squirrel = wn.synsets('squirrel')[0]
weasel = wn.synsets('weasel')[1] # index 0 is a sneaky person
squirrel.lowest_common_hypernyms(weasel)

snake = wn.synsets('snake')[0]
print(snake.lowest_common_hypernyms(weasel))
print(snake.lowest_common_hypernyms(squirrel))

turtle = wn.synsets('turtle')[1] # index 0 is a turtleneck (lmao)
print(turtle.lowest_common_hypernyms(snake))
print(turtle.lowest_common_hypernyms(squirrel))

fish = wn.synsets('fish', pos=wn.NOUN)[0] # pos = part of speech
swan = wn.synsets('swan')[0]
print(fish.lowest_common_hypernyms(turtle))
print(fish.lowest_common_hypernyms(snake))

louse = wn.synsets('louse')[0]
oyster = wn.synsets('oyster')[0]
print(louse.lowest_common_hypernyms(snake))
print(louse.lowest_common_hypernyms(oyster))

[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('reptile.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('vertebrate.n.01')]
[Synset('animal.n.01')]
[Synset('invertebrate.n.01')]


In [33]:
class Node(object):

    def __init__(self, value, parent, children =[]):
        self.value = value
        self.children = children
        if parent is not None:
            parent.children.append(self)

In [34]:
from print_tree import print_tree

class print_custom_tree(print_tree):

    def get_children(self, node):
        return node.children

    def get_node_str(self, node):
        return str(node.value)

In [35]:
animaln = Node(wn.synsets('animal')[0], None)
visited = []
c=0

for animals in simple_terms[0:5] :
    if animals not in ['calendra','bombicum','water-dog','og','turtledofe','calandra','verdaule','tellin','petit-gri','aucupio','daot','shell','hart','mutton','dragon','mussel','barbel','codfish','crayfish','pork','human']:   #not in wordnet or not in tree
        if animals in ['weasel','hog','partridge','chicken','turtle','cuckoo']: #first meaning is not the animal
            ani = wn.synsets(animals)[1]
        elif animals in ['ewe','swallow']:  #have to go to third meaning to get the animal
             ani = wn.synsets(animals)[2]  
        else :
            ani = wn.synsets(animals)[0]
        an = r_node(ani)
        lis = an.return_animal_path()[0]
        for i in range (1,len(lis)):
            name = lis[i].name().split('.')[0]
            print(name)
            if name in ['even-toed_ungulate','odd-toed_ungulate','orthopterous_insect','chordate','placental','ungulate','thrush','oscine','leporid','lagomorph','decapod_crustacean','chordate']: #- make everything go boom also things I don't care about
                c+=1
                visited.append(name)
            elif name not in visited :
                name_prec = lis[i-1-c].name().split('.')[0]
                c=0
                string = name+'n'+' = Node(name,'+ name_prec+'n)'
                exec(string)
                visited.append(name)
            else :
                c=0

#print_custom_tree(animaln)  #can't do it, it explodes 

chordate
vertebrate
bird
passerine
chordate
vertebrate
bird
chordate
vertebrate
mammal
placental
carnivore
canine
dog
